In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

print("="*80)
print("PROCESAMIENTO COMPLETO ENDES 2015-2024")
print("GENERACIÓN DE BASES: TRAIN / TEST / DEV")
print("="*80)

# ============================================================
# CONFIGURACIÓN
# ============================================================
ruta_base = r"D:"
ruta_salida = r"D:\Bases_train_test"
años = list(range(2015, 2025))  # 2015 a 2024

# Variables a conservar por cada base
VARIABLES_ESENCIALES = {
    'RECH6': [
        'HHID', 'HC0', 'HC51',  # Identificadores
        'HC1',   # Edad en meses
        'HC27',  # Sexo
        'HC53',  # Hemoglobina (g/L)
        'HC55',  # Resultado de medición
        'HC57',  # Nivel de anemia
        'HC70',  # Altura/edad Z-score
    ],
    
    'RECH1': [
        'HHID', 'HVIDX',  # Identificadores
        'HV103',  # Durmió noche anterior
        'HV009',  # Número de miembros del hogar
    ],
    
    'RECH0': [
        'HHID',    # Identificador
        'HV001',   # Conglomerado
        'HV002',   # Número de hogar
        'HV005',   # Factor de ponderación
        'HV005X',  # Factor de ponderación (alternativo)
        'HV007',   # Año de entrevista
        'HV015',   # Resultado del cuestionario
        'HV024',   # Región/departamento
        'HV025',   # Área de residencia
        'HV040',   # Altitud
        'HV201',   # Fuente de agua
        'HV205',   # Tipo de servicio sanitario
        'HV206',   # Electricidad
        'HV271',   # Índice de riqueza
    ],
    
    'RECH23': [
        'HHID',    # Identificador
        'HV237',   # Tratamiento del agua
    ],
    
    'REC44': [
        'CASEID', 'HWIDX',  # Identificadores
        'HW1',   # Edad en meses
        'HW2',   # Peso (kg)
        'HW3',   # Talla (cm)
        'HW70',  # Talla/Edad Z-score
        'HW71',  # Peso/Edad Z-score
        'HW72',  # Peso/Talla Z-score
        'HW73',  # IMC Z-score
    ],
    
    'REC21': [
        'CASEID', 'B16', 'BIDX',  # Identificadores
        'BORD',  # Orden de nacimiento
    ],
    
    'REC0111': [
        'CASEID',  # Identificador
        'V012',    # Edad de la madre
        'V025',    # Área de residencia
        'V106',    # Nivel educativo de la madre
        'V133',    # Educación en años
        'V190',    # Índice de riqueza
    ]
}

# ============================================================
# FUNCIONES DE LIMPIEZA
# ============================================================
def limpiar_ids(df, cols):
    """Limpia identificadores eliminando .0, espacios al inicio/final"""
    for col in cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(r'\.0$', '', regex=True)
            df[col] = df[col].str.strip()
    return df

def limpiar_caseid(caseid_series):
    """Limpieza específica para CASEID: normaliza a doble espacio"""
    caseid_clean = caseid_series.astype(str)
    caseid_clean = caseid_clean.str.replace(r'\.0$', '', regex=True)
    caseid_clean = caseid_clean.str.strip()
    caseid_clean = caseid_clean.str.replace(r'\s{2,}', '  ', regex=True)
    return caseid_clean

def procesar_año(año):
    """Procesa un año completo de ENDES"""
    
    print(f"\n{'='*80}")
    print(f"PROCESANDO AÑO {año}")
    print(f"{'='*80}")
    
    ruta_datos = Path(ruta_base) / str(año)
    
    # 1. CARGAR BASES
    print(f"\n1. Cargando bases de datos...")
    bases = {}
    lista_bases = ['RECH0', 'RECH1', 'RECH6', 'RECH23', 'REC0111', 'REC21', 'REC44']
    
    for nombre_base in lista_bases:
        try:
            archivo = ruta_datos / f"{nombre_base}.sav"
            
            if nombre_base in VARIABLES_ESENCIALES:
                df_temp = pd.read_spss(str(archivo))
                cols_disponibles = df_temp.columns.tolist()
                
                cols_a_cargar = [col for col in VARIABLES_ESENCIALES[nombre_base] if col in cols_disponibles]
                cols_faltantes = [col for col in VARIABLES_ESENCIALES[nombre_base] if col not in cols_disponibles]
                
                df = df_temp[cols_a_cargar].copy()
                bases[nombre_base] = df
                
                print(f"   ✓ {nombre_base:10s}: {df.shape[0]:>6,} registros, {df.shape[1]:>3} columnas")
                if cols_faltantes:
                    print(f"      ⚠️  No encontradas: {', '.join(cols_faltantes[:3])}")
                    
        except FileNotFoundError:
            print(f"   ✗ {nombre_base:10s}: Archivo no encontrado")
        except Exception as e:
            print(f"   ✗ {nombre_base:10s}: Error - {str(e)[:50]}")
    
    if 'RECH6' not in bases:
        print(f"\n❌ ERROR: No se pudo cargar RECH6 para {año}")
        return None
    
    # 2. LIMPIAR IDENTIFICADORES
    print(f"\n2. Limpiando identificadores...")
    
    # RECH6
    bases['RECH6'] = limpiar_ids(bases['RECH6'], ['HHID', 'HC0', 'HC51'])
    bases['RECH6']['CASEID'] = (
        bases['RECH6']['HHID'].astype(str).str.strip() + "  " +
        bases['RECH6']['HC51'].astype(str).str.strip()
    )
    bases['RECH6']['CASEID'] = limpiar_caseid(bases['RECH6']['CASEID'])
    bases['RECH6']['ANIO'] = año
    print(f"   ✓ RECH6: CASEID construido, ANIO agregado")
    
    # Otras bases
    if 'RECH0' in bases:
        bases['RECH0'] = limpiar_ids(bases['RECH0'], ['HHID'])
    if 'RECH1' in bases:
        bases['RECH1'] = limpiar_ids(bases['RECH1'], ['HHID', 'HVIDX'])
    if 'RECH23' in bases:
        bases['RECH23'] = limpiar_ids(bases['RECH23'], ['HHID'])
    if 'REC44' in bases:
        bases['REC44'] = limpiar_ids(bases['REC44'], ['HWIDX'])
        if 'CASEID' in bases['REC44'].columns:
            bases['REC44']['CASEID'] = limpiar_caseid(bases['REC44']['CASEID'])
    if 'REC21' in bases:
        bases['REC21'] = limpiar_ids(bases['REC21'], ['BIDX', 'B16'])
        if 'CASEID' in bases['REC21'].columns:
            bases['REC21']['CASEID'] = limpiar_caseid(bases['REC21']['CASEID'])
    if 'REC0111' in bases:
        if 'CASEID' in bases['REC0111'].columns:
            bases['REC0111']['CASEID'] = limpiar_caseid(bases['REC0111']['CASEID'])
    
    # 3. APLICAR FILTROS
    print(f"\n3. Aplicando filtros metodológicos...")
    
    # Filtros RECH6
    n_antes = len(bases['RECH6'])
    filtros_rech6 = (
        (bases['RECH6']['HC1'] >= 6) & 
        (bases['RECH6']['HC1'] <= 35) &
        (bases['RECH6']['HC55'].isin([0.0, 'Medido', 'Measured']))
    )
    bases['RECH6'] = bases['RECH6'][filtros_rech6].copy()
    print(f"   RECH6: {n_antes:,} → {len(bases['RECH6']):,} (edad 6-35 meses, medición válida)")
    
    # Filtros RECH1
    if 'RECH1' in bases and 'HV103' in bases['RECH1'].columns:
        n_antes = len(bases['RECH1'])
        bases['RECH1']['HV103'] = bases['RECH1']['HV103'].astype(str).str.strip().str.lower()
        filtros_rech1 = bases['RECH1']['HV103'].isin(['sí', 'si', 'yes', '1', '1.0'])
        bases['RECH1'] = bases['RECH1'][filtros_rech1].copy()
        print(f"   RECH1: {n_antes:,} → {len(bases['RECH1']):,} (durmió noche anterior)")
    
    # Filtros RECH0
    if 'RECH0' in bases and 'HV015' in bases['RECH0'].columns:
        n_antes = len(bases['RECH0'])
        filtros_rech0 = bases['RECH0']['HV015'].isin([1.0, 'Completo', 'Completed','Completa'])
        bases['RECH0'] = bases['RECH0'][filtros_rech0].copy()
        print(f"   RECH0: {n_antes:,} → {len(bases['RECH0']):,} (cuestionario completo)")
    
    # 4. REALIZAR JOINS
    print(f"\n4. Realizando joins...")
    
    df = bases['RECH6'].copy()
    print(f"   Base inicial: {len(df):,} registros")
    
    # JOIN 1: RECH1
    if 'RECH1' in bases:
        n_antes = len(df)
        df = pd.merge(df, bases['RECH1'],
                     left_on=['HHID', 'HC0'],
                     right_on=['HHID', 'HVIDX'],
                     how='left',
                     suffixes=('', '_R1'))
        n_matched = (df['HVIDX'].notna()).sum()
        print(f"   ✓ JOIN 1: RECH1 | Matched: {n_matched:,} ({n_matched/n_antes*100:.1f}%)")
    
    # JOIN 2: RECH0
    if 'RECH0' in bases:
        n_antes = len(df)
        cols_rech0 = [c for c in bases['RECH0'].columns if c not in df.columns or c == 'HHID']
        df = pd.merge(df, bases['RECH0'][cols_rech0],
                     on='HHID',
                     how='left',
                     suffixes=('', '_R0'))
        n_matched = (df['HV007'].notna()).sum() if 'HV007' in df.columns else 0
        print(f"   ✓ JOIN 2: RECH0 | Matched: {n_matched:,} ({n_matched/n_antes*100:.1f}%)")
    
    # JOIN 3: RECH23
    if 'RECH23' in bases:
        n_antes = len(df)
        cols_rech23 = [c for c in bases['RECH23'].columns if c not in df.columns or c == 'HHID']
        df = pd.merge(df, bases['RECH23'][cols_rech23],
                     on='HHID',
                     how='left',
                     suffixes=('', '_R23'))
        col_check = 'HV237' if 'HV237' in df.columns else None
        n_matched = (df[col_check].notna()).sum() if col_check else 0
        print(f"   ✓ JOIN 3: RECH23 | Matched: {n_matched:,} ({n_matched/n_antes*100:.1f}%)")
    
    # JOIN 4: REC44 (SOLO CASEID)
    if 'REC44' in bases and 'CASEID' in df.columns:
        n_antes = len(df)
        cols_rec44 = [c for c in bases['REC44'].columns if c not in df.columns or c == 'CASEID']
        df = pd.merge(df, bases['REC44'][cols_rec44],
                     on='CASEID',
                     how='left',
                     suffixes=('', '_R44'))
        n_matched = (df['HW1'].notna()).sum() if 'HW1' in df.columns else 0
        print(f"   ✓ JOIN 4: REC44  | Matched: {n_matched:,} ({n_matched/n_antes*100:.1f}%)")
        if len(df) > n_antes:
            print(f"      ⚠️  Duplicación: +{len(df)-n_antes} registros")
    
    # JOIN 5: REC21
    if 'REC21' in bases and 'CASEID' in df.columns:
        n_antes = len(df)
        if 'B16' in bases['REC21'].columns:
            cols_rec21 = [c for c in bases['REC21'].columns if c not in df.columns or c in ['CASEID', 'B16']]
            df = pd.merge(df, bases['REC21'][cols_rec21],
                         left_on=['CASEID', 'HC0'],
                         right_on=['CASEID', 'B16'],
                         how='left',
                         suffixes=('', '_REC21'))
            n_matched = (df['BORD'].notna()).sum() if 'BORD' in df.columns else 0
            print(f"   ✓ JOIN 5: REC21  | Matched: {n_matched:,} ({n_matched/n_antes*100:.1f}%)")
    
    # JOIN 6: REC0111
    if 'REC0111' in bases and 'CASEID' in df.columns:
        n_antes = len(df)
        cols_rec0111 = [c for c in bases['REC0111'].columns if c not in df.columns or c == 'CASEID']
        df = pd.merge(df, bases['REC0111'][cols_rec0111],
                     on='CASEID',
                     how='left',
                     suffixes=('', '_REC0111'))
        n_matched = (df['V012'].notna()).sum() if 'V012' in df.columns else 0
        print(f"   ✓ JOIN 6: REC0111 | Matched: {n_matched:,} ({n_matched/n_antes*100:.1f}%)")
    
    # 5. CREAR VARIABLE ANEMIA
    print(f"\n5. Creando variable ANEMIA...")
    
    if 'HC57' in df.columns:
        df['HC57_str'] = df['HC57'].astype(str).str.strip().str.lower()
        df['ANEMIA'] = df['HC57_str'].apply(
            lambda x: 0 if x in ['sin anemia', 'not anemic','no anémico'] else (
                1 if x in ['leve', 'moderada', 'grave', 'mild', 'moderate', 'severe','severa','moderar'] else np.nan
            )
        )
        n_anemia = (df['ANEMIA']==1).sum()
        pct_anemia = n_anemia / len(df) * 100
        print(f"   ✓ Con anemia: {n_anemia:,} ({pct_anemia:.1f}%)")
        print(f"   ✓ Sin anemia: {(df['ANEMIA']==0).sum():,} ({(df['ANEMIA']==0).sum()/len(df)*100:.1f}%)")
    
    print(f"\n✅ Año {año} procesado: {len(df):,} registros × {df.shape[1]} columnas")
    
    return df

# ============================================================
# PROCESAMIENTO DE TODOS LOS AÑOS
# ============================================================
print("\n" + "="*80)
print("PROCESANDO TODOS LOS AÑOS")
print("="*80)

datasets = []
resumen_años = []

for año in años:
    df_año = procesar_año(año)
    
    if df_año is not None:
        datasets.append(df_año)
        
        resumen_años.append({
            'Año': año,
            'Registros': len(df_año),
            'Con_Anemia': (df_año['ANEMIA']==1).sum() if 'ANEMIA' in df_año.columns else 0,
            'Pct_Anemia': (df_año['ANEMIA']==1).sum() / len(df_año) * 100 if 'ANEMIA' in df_año.columns else 0,
            'Columnas': df_año.shape[1]
        })

if not datasets:
    print("\n❌ ERROR: No se procesó ningún año correctamente")
    exit(1)

# ============================================================
# CONSOLIDACIÓN
# ============================================================
print("\n" + "="*80)
print("CONSOLIDANDO TODOS LOS AÑOS")
print("="*80)

df_consolidado = pd.concat(datasets, ignore_index=True, sort=False)

print(f"\n📊 Dataset consolidado:")
print(f"   - Total registros: {len(df_consolidado):,}")
print(f"   - Total columnas: {df_consolidado.shape[1]}")
print(f"   - Años: {sorted(df_consolidado['ANIO'].unique())}")

# ============================================================
# HOMOLOGACIÓN DE VARIABLES CATEGÓRICAS
# ============================================================
print("\n" + "="*80)
print("HOMOLOGANDO VARIABLES CATEGÓRICAS")
print("="*80)

# HC51: No en el hogar → MISSING
if 'HC51' in df_consolidado.columns:
    df_consolidado['HC51'] = df_consolidado['HC51'].replace({
        'No en el hogar': np.nan,
        'Not in household': np.nan
    })
    print(f"   ✓ HC51: Convertido 'No en el hogar' a MISSING")

# HC27: Sexo → Estandarizar a español
if 'HC27' in df_consolidado.columns:
    df_consolidado['HC27'] = df_consolidado['HC27'].astype(str).str.strip().str.lower()
    df_consolidado['HC27'] = df_consolidado['HC27'].replace({
        'female': 'mujer',
        'male': 'hombre',
        'hombre': 'hombre',
        'mujer': 'mujer'
    })
    print(f"   ✓ HC27: Convertido a Hombre/Mujer")

# HC55: Resultado de medición → Estandarizar a español
if 'HC55' in df_consolidado.columns:
    df_consolidado['HC55'] = df_consolidado['HC55'].astype(str).str.strip().str.lower()
    df_consolidado['HC55'] = df_consolidado['HC55'].replace({
        'measured': 'medido',
        'medido': 'medido'
    })
    print(f"   ✓ HC55: Convertido a Medido")

# HC57: Nivel de anemia → Estandarizar a español
if 'HC57' in df_consolidado.columns:
    df_consolidado['HC57'] = df_consolidado['HC57'].astype(str).str.strip().str.lower()
    df_consolidado['HC57'] = df_consolidado['HC57'].replace({
        'grave': 'grave',
        'severe': 'grave',
        'severa': 'grave',
        'leve': 'leve',
        'mild': 'leve',
        'moderada': 'moderada',
        'moderar': 'moderada',
        'moderate': 'moderada',
        'no anémico': 'sin anemia',
        'not anemic': 'sin anemia',
        'sin anemia': 'sin anemia'
    })
    print(f"   ✓ HC57: Estandarizado (Grave, Leve, Moderada, Sin anemia)")

# HC70, HW70, HW71, HW72, HW73: Convertir a numérico (si tiene letras → missing)
for var in ['HC70', 'HW70', 'HW71', 'HW72', 'HW73']:
    if var in df_consolidado.columns:
        # Convertir a numérico, los valores no numéricos quedan como NaN
        df_consolidado[var] = pd.to_numeric(df_consolidado[var], errors='coerce')
        print(f"   ✓ {var}: Convertido a numérico (letras → missing)")

# HV103: Durmió noche anterior → Estandarizar a sí/no
if 'HV103' in df_consolidado.columns:
    df_consolidado['HV103'] = df_consolidado['HV103'].astype(str).str.strip().str.lower()
    df_consolidado['HV103'] = df_consolidado['HV103'].replace({
        'si': 'sí',
        'sí': 'sí',
        'yes': 'sí'
    })
    print(f"   ✓ HV103: Convertido a Sí")

# HV015: Resultado cuestionario → Estandarizar a español
if 'HV015' in df_consolidado.columns:
    df_consolidado['HV015'] = df_consolidado['HV015'].astype(str).str.strip().str.lower()
    df_consolidado['HV015'] = df_consolidado['HV015'].replace({
        'completed': 'completo',
        'completo': 'completo'
    })
    print(f"   ✓ HV015: Convertido a Completo")

# HV025: Área de residencia → Solo Rural/Urbano
if 'HV025' in df_consolidado.columns:
    df_consolidado['HV025'] = df_consolidado['HV025'].astype(str).str.strip().str.lower()
    df_consolidado['HV025'] = df_consolidado['HV025'].replace({
        'rural': 'rural',
        'urban': 'urbano',
        'urbano': 'urbano',
        'urbana': 'urbano'
    })
    print(f"   ✓ HV025: Estandarizado (Rural, Urbano)")

# HV237: Tratamiento agua → Solo Sí/No
if 'HV237' in df_consolidado.columns:
    df_consolidado['HV237'] = df_consolidado['HV237'].astype(str).str.strip().str.lower()
    df_consolidado['HV237'] = df_consolidado['HV237'].replace({
        'no': 'no',
        'si': 'sí',
        'sí': 'sí',
        'yes': 'sí'
    })
    print(f"   ✓ HV237: Convertido a Sí/No")

# V106: Nivel educativo → Estandarizar a español
if 'V106' in df_consolidado.columns:
    df_consolidado['V106'] = df_consolidado['V106'].astype(str).str.strip().str.lower()
    df_consolidado['V106'] = df_consolidado['V106'].replace({
        'higher': 'superior',
        'mayor': 'superior',
        'no education': 'sin educación',
        'sin educación': 'sin educación',
        'primaria': 'primaria',
        'primario': 'primaria',
        'primary': 'primaria',
        'secondary': 'secundaria',
        'secundaria': 'secundaria',
        'secundario': 'secundaria',
        'superior': 'superior',
        '(en blanco)': np.nan
    })
    print(f"   ✓ V106: Estandarizado (Sin educación, Primaria, Secundaria, Superior)")

# V190: Índice de riqueza → Homologar a español
if 'V190' in df_consolidado.columns:
    df_consolidado['V190'] = df_consolidado['V190'].astype(str).str.strip().str.lower()
    df_consolidado['V190'] = df_consolidado['V190'].replace({
        'el más pobre': 'más pobre',
        'poorest': 'más pobre',
        'poorer': 'pobre',
        'pobre': 'pobre',
        'pobrer': 'pobre',
        'medio': 'medio',
        'middle': 'medio',
        'más rico': 'más rico',
        'richer': 'rico',
        'rico': 'rico',
        'richest': 'más rico',
        'muy pobre': 'más pobre',
        'muy rico': 'más rico'
    })
    print(f"   ✓ V190: Estandarizado (Más pobre, Pobre, Medio, Rico, Más rico)")

# V025: Área de residencia (puede venir de REC0111 también)
if 'V025' in df_consolidado.columns:
    df_consolidado['V025'] = df_consolidado['V025'].astype(str).str.strip().str.lower()
    df_consolidado['V025'] = df_consolidado['V025'].replace({
        'rural': 'rural',
        'urban': 'urbano',
        'urbano': 'urbano',
        'urbana': 'urbano'
    })
    print(f"   ✓ V025: Estandarizado (Rural, Urbano)")

print(f"\n✅ Homologación completada")

# Crear variable PESO
if 'HV005' in df_consolidado.columns and 'HV007' in df_consolidado.columns:
    df_consolidado['PESO'] = np.where(
        df_consolidado['HV007'] == 2015,
        df_consolidado['HV005X'] / 1_000_000 if 'HV005X' in df_consolidado.columns else df_consolidado['HV005'] / 1_000_000,
        df_consolidado['HV005'] / 1_000_000
    )
    print(f"\n✓ Variable PESO creada")

# ============================================================
# PARTICIÓN DE DATOS: TRAIN / TEST / DEV CON ESTRATIFICACIÓN CRUZADA
# ============================================================
from sklearn.model_selection import train_test_split

print("\n" + "="*80)
print("PARTICIÓN DE DATOS: TRAIN / TEST / DEV CON ESTRATIFICACIÓN CRUZADA")
print("="*80)

print("""
📊 ESTRATEGIA DE MUESTREO ESTRATIFICADO CRUZADO:
   - TRAIN: 70% de CADA año (2015-2024)
   - TEST:  20% de CADA año (2015-2024)
   - DEV:   10% de CADA año (2015-2024)
   
   ✓ Estratificación por AÑO × ANEMIA
   ✓ Cada partición tiene TODOS los años
   ✓ Proporción de anemia por año se mantiene
   ✓ Ejemplo: Si 2015 tiene 40% anemia, TRAIN/TEST/DEV también
""")

# Filtrar solo registros con ANEMIA válida (0 o 1)
df_valido = df_consolidado[df_consolidado['ANEMIA'].isin([0, 1])].copy()
n_excluidos = len(df_consolidado) - len(df_valido)

if n_excluidos > 0:
    print(f"\n⚠️  Excluidos {n_excluidos:,} registros con ANEMIA = NaN")

# Crear variable de estratificación: ANIO_ANEMIA
df_valido['ESTRATO'] = df_valido['ANIO'].astype(str) + '_' + df_valido['ANEMIA'].astype(int).astype(str)

print(f"\n📋 Base consolidada: {len(df_valido):,} registros")
print(f"   Años: {sorted(df_valido['ANIO'].unique())}")
print(f"   Proporción global ANEMIA: {(df_valido['ANEMIA']==1).sum()/len(df_valido)*100:.2f}%")

# Mostrar distribución por estrato
print(f"\n🔍 Distribución por estrato (AÑO × ANEMIA):")
estrato_counts = df_valido.groupby(['ANIO', 'ANEMIA']).size().reset_index(name='N')
estrato_counts['Porcentaje'] = (estrato_counts['N'] / len(df_valido) * 100).round(2)

print(f"\n   {'Año':<6} {'Anemia':<10} {'Registros':>10} {'%':>8}")
print(f"   {'-'*6} {'-'*10} {'-'*10} {'-'*8}")
for idx, row in estrato_counts.iterrows():
    anemia_label = 'Con' if row['ANEMIA'] == 1 else 'Sin'
    print(f"   {int(row['ANIO']):<6} {anemia_label:<10} {int(row['N']):>10,} {row['Porcentaje']:>7.2f}%")

# Verificar que todos los estratos tengan suficientes casos
print(f"\n⚙️  Verificando estratos...")
min_casos_por_estrato = estrato_counts['N'].min()
if min_casos_por_estrato < 10:
    print(f"   ⚠️  ADVERTENCIA: Algunos estratos tienen < 10 casos")
    print(f"   Mínimo: {min_casos_por_estrato} casos")
else:
    print(f"   ✓ Todos los estratos tienen ≥ 10 casos (mínimo: {min_casos_por_estrato})")

# ============================================================
# PASO 1: Dividir en TRAIN+TEST (90%) y DEV (10%)
# ============================================================
print(f"\n{'='*80}")
print("PASO 1: Separar DEV (10%)")
print(f"{'='*80}")

df_train_test, df_dev = train_test_split(
    df_valido,
    test_size=0.10,
    stratify=df_valido['ESTRATO'],
    random_state=42
)

print(f"\n✓ TRAIN+TEST: {len(df_train_test):,} registros (90%)")
print(f"✓ DEV:        {len(df_dev):,} registros (10%)")

# Verificar estratificación en DEV
print(f"\n   DEV - Distribución por año:")
for año in sorted(df_dev['ANIO'].unique()):
    n_año = len(df_dev[df_dev['ANIO'] == año])
    n_anemia = (df_dev['ANEMIA'][df_dev['ANIO'] == año] == 1).sum()
    pct_anemia = n_anemia / n_año * 100 if n_año > 0 else 0
    print(f"      {int(año)}: {n_año:>5,} registros ({pct_anemia:.1f}% con anemia)")

# ============================================================
# PASO 2: Dividir TRAIN+TEST en TRAIN (70%) y TEST (20%)
# ============================================================
print(f"\n{'='*80}")
print("PASO 2: Dividir TRAIN (70%) y TEST (20%)")
print(f"{'='*80}")

# 70% del total = 70/90 = 0.7778 de train_test
df_train, df_test = train_test_split(
    df_train_test,
    test_size=0.2222,  # 20% del total / 90% = 0.2222
    stratify=df_train_test['ESTRATO'],
    random_state=42
)

print(f"\n✓ TRAIN: {len(df_train):,} registros (70%)")
print(f"✓ TEST:  {len(df_test):,} registros (20%)")

# Verificar estratificación en TRAIN
print(f"\n   TRAIN - Distribución por año:")
for año in sorted(df_train['ANIO'].unique()):
    n_año = len(df_train[df_train['ANIO'] == año])
    n_anemia = (df_train['ANEMIA'][df_train['ANIO'] == año] == 1).sum()
    pct_anemia = n_anemia / n_año * 100 if n_año > 0 else 0
    print(f"      {int(año)}: {n_año:>5,} registros ({pct_anemia:.1f}% con anemia)")

# Verificar estratificación en TEST
print(f"\n   TEST - Distribución por año:")
for año in sorted(df_test['ANIO'].unique()):
    n_año = len(df_test[df_test['ANIO'] == año])
    n_anemia = (df_test['ANEMIA'][df_test['ANIO'] == año] == 1).sum()
    pct_anemia = n_anemia / n_año * 100 if n_año > 0 else 0
    print(f"      {int(año)}: {n_año:>5,} registros ({pct_anemia:.1f}% con anemia)")

# ============================================================
# VERIFICACIÓN FINAL
# ============================================================
total_registros = len(df_valido)

print("\n" + "="*80)
print("📊 RESUMEN FINAL DE PARTICIÓN")
print("="*80)

print(f"\n🎯 TRAIN: {len(df_train):,} registros ({len(df_train)/total_registros*100:.1f}%)")
print(f"   - Con anemia:    {(df_train['ANEMIA']==1).sum():>6,} ({(df_train['ANEMIA']==1).sum()/len(df_train)*100:.2f}%)")
print(f"   - Sin anemia:    {(df_train['ANEMIA']==0).sum():>6,} ({(df_train['ANEMIA']==0).sum()/len(df_train)*100:.2f}%)")
print(f"   - Años únicos:   {sorted(df_train['ANIO'].unique())}")

print(f"\n🎯 TEST: {len(df_test):,} registros ({len(df_test)/total_registros*100:.1f}%)")
print(f"   - Con anemia:    {(df_test['ANEMIA']==1).sum():>6,} ({(df_test['ANEMIA']==1).sum()/len(df_test)*100:.2f}%)")
print(f"   - Sin anemia:    {(df_test['ANEMIA']==0).sum():>6,} ({(df_test['ANEMIA']==0).sum()/len(df_test)*100:.2f}%)")
print(f"   - Años únicos:   {sorted(df_test['ANIO'].unique())}")

print(f"\n🎯 DEV: {len(df_dev):,} registros ({len(df_dev)/total_registros*100:.1f}%)")
print(f"   - Con anemia:    {(df_dev['ANEMIA']==1).sum():>6,} ({(df_dev['ANEMIA']==1).sum()/len(df_dev)*100:.2f}%)")
print(f"   - Sin anemia:    {(df_dev['ANEMIA']==0).sum():>6,} ({(df_dev['ANEMIA']==0).sum()/len(df_dev)*100:.2f}%)")
print(f"   - Años únicos:   {sorted(df_dev['ANIO'].unique())}")

# Verificar balance entre particiones
print("\n" + "="*80)
print("✓ VERIFICACIÓN DE ESTRATIFICACIÓN CRUZADA (AÑO × ANEMIA)")
print("="*80)

prop_anemia_train = (df_train['ANEMIA']==1).sum() / len(df_train) * 100
prop_anemia_test = (df_test['ANEMIA']==1).sum() / len(df_test) * 100
prop_anemia_dev = (df_dev['ANEMIA']==1).sum() / len(df_dev) * 100
prop_anemia_global = (df_valido['ANEMIA']==1).sum() / len(df_valido) * 100

print(f"\n📊 Proporción GLOBAL de ANEMIA:")
print(f"   GLOBAL: {prop_anemia_global:.2f}%")
print(f"   TRAIN:  {prop_anemia_train:.2f}% (Δ: {abs(prop_anemia_train - prop_anemia_global):.2f}pp)")
print(f"   TEST:   {prop_anemia_test:.2f}% (Δ: {abs(prop_anemia_test - prop_anemia_global):.2f}pp)")
print(f"   DEV:    {prop_anemia_dev:.2f}% (Δ: {abs(prop_anemia_dev - prop_anemia_global):.2f}pp)")

# Verificar distribución por año
print(f"\n📊 Proporción de ANEMIA por AÑO (comparativa):")
print(f"\n   {'Año':<6} {'Original':>10} {'TRAIN':>10} {'TEST':>10} {'DEV':>10}")
print(f"   {'-'*6} {'-'*10} {'-'*10} {'-'*10} {'-'*10}")

for año in sorted(df_valido['ANIO'].unique()):
    # Original
    df_año_orig = df_valido[df_valido['ANIO'] == año]
    pct_orig = (df_año_orig['ANEMIA'] == 1).sum() / len(df_año_orig) * 100 if len(df_año_orig) > 0 else 0
    
    # TRAIN
    df_año_train = df_train[df_train['ANIO'] == año]
    pct_train = (df_año_train['ANEMIA'] == 1).sum() / len(df_año_train) * 100 if len(df_año_train) > 0 else 0
    
    # TEST
    df_año_test = df_test[df_test['ANIO'] == año]
    pct_test = (df_año_test['ANEMIA'] == 1).sum() / len(df_año_test) * 100 if len(df_año_test) > 0 else 0
    
    # DEV
    df_año_dev = df_dev[df_dev['ANIO'] == año]
    pct_dev = (df_año_dev['ANEMIA'] == 1).sum() / len(df_año_dev) * 100 if len(df_año_dev) > 0 else 0
    
    print(f"   {int(año):<6} {pct_orig:>9.2f}% {pct_train:>9.2f}% {pct_test:>9.2f}% {pct_dev:>9.2f}%")

# Verificar diferencias máximas
max_diff = max(
    abs(prop_anemia_train - prop_anemia_global),
    abs(prop_anemia_test - prop_anemia_global),
    abs(prop_anemia_dev - prop_anemia_global)
)

print(f"\n{'='*80}")
if max_diff < 0.5:
    print(f"✅ Estratificación EXCELENTE: diferencia máxima < 0.5pp ({max_diff:.2f}pp)")
elif max_diff < 1:
    print(f"✅ Estratificación MUY BUENA: diferencia máxima < 1pp ({max_diff:.2f}pp)")
elif max_diff < 2:
    print(f"✓ Estratificación BUENA: diferencia máxima < 2pp ({max_diff:.2f}pp)")
else:
    print(f"⚠️  Estratificación ACEPTABLE: diferencia máxima = {max_diff:.2f}pp")
print(f"{'='*80}")

# ============================================================
# GUARDAR RESULTADOS
# ============================================================
print("\n" + "="*80)
print("GUARDANDO RESULTADOS")
print("="*80)

# Crear carpeta de salida si no existe
Path(ruta_salida).mkdir(parents=True, exist_ok=True)

# Guardar bases particionadas
archivo_train = Path(ruta_salida) / "endes_train_2015_2024.csv"
df_train.to_csv(archivo_train, index=False, encoding='utf-8-sig')
print(f"\n✓ TRAIN:  {archivo_train}")
print(f"   {len(df_train):,} registros × {df_train.shape[1]} columnas")
print(f"   Incluye todos los años (2015-2024) en proporción 70%")

archivo_test = Path(ruta_salida) / "endes_test_2015_2024.csv"
df_test.to_csv(archivo_test, index=False, encoding='utf-8-sig')
print(f"\n✓ TEST:   {archivo_test}")
print(f"   {len(df_test):,} registros × {df_test.shape[1]} columnas")
print(f"   Incluye todos los años (2015-2024) en proporción 20%")

archivo_dev = Path(ruta_salida) / "endes_dev_2015_2024.csv"
df_dev.to_csv(archivo_dev, index=False, encoding='utf-8-sig')
print(f"\n✓ DEV:    {archivo_dev}")
print(f"   {len(df_dev):,} registros × {df_dev.shape[1]} columnas")
print(f"   Incluye todos los años (2015-2024) en proporción 10%")

# Guardar también base consolidada completa
archivo_completo = Path(ruta_salida) / "endes_2015_2024_consolidado.csv"
df_consolidado.to_csv(archivo_completo, index=False, encoding='utf-8-sig')
print(f"\n✓ COMPLETO: {archivo_completo}")
print(f"   {len(df_consolidado):,} registros × {df_consolidado.shape[1]} columnas")

# ============================================================
# GENERAR REPORTE EXCEL
# ============================================================
print("\n" + "="*80)
print("GENERANDO REPORTE EXCEL")
print("="*80)

archivo_resumen = Path(ruta_salida) / "resumen_particion_endes.xlsx"

with pd.ExcelWriter(archivo_resumen, engine='openpyxl') as writer:
    
    # Hoja 1: Resumen por año
    df_resumen = pd.DataFrame(resumen_años)
    df_resumen.to_excel(writer, sheet_name='Resumen_Años', index=False)
    
    # Hoja 2: Variables disponibles
    vars_disponibles = pd.DataFrame({
        'Variable': df_consolidado.columns,
        'Tipo': df_consolidado.dtypes.astype(str),
        'No_Nulos': [df_consolidado[col].notna().sum() for col in df_consolidado.columns],
        'Porcentaje': [f"{df_consolidado[col].notna().sum()/len(df_consolidado)*100:.1f}%" for col in df_consolidado.columns]
    })
    vars_disponibles.to_excel(writer, sheet_name='Variables', index=False)
    
    # Hoja 3: Estadísticas de partición con estratificación
    stats_particion = pd.DataFrame({
        'Dataset': ['TRAIN (2015-2021)', 'TEST (2022-2023)', 'DEV (2024)', 'TOTAL'],
        'Registros': [
            len(df_train), 
            len(df_test), 
            len(df_dev), 
            len(df_valido)
        ],
        'Porcentaje': [
            f"{len(df_train)/total_registros*100:.1f}%",
            f"{len(df_test)/total_registros*100:.1f}%",
            f"{len(df_dev)/total_registros*100:.1f}%",
            "100.0%"
        ],
        'Con_Anemia': [
            (df_train['ANEMIA']==1).sum(),
            (df_test['ANEMIA']==1).sum(),
            (df_dev['ANEMIA']==1).sum(),
            (df_valido['ANEMIA']==1).sum()
        ],
        'Pct_Anemia': [
            f"{(df_train['ANEMIA']==1).sum() / len(df_train) * 100:.2f}%",
            f"{(df_test['ANEMIA']==1).sum() / len(df_test) * 100:.2f}%",
            f"{(df_dev['ANEMIA']==1).sum() / len(df_dev) * 100:.2f}%",
            f"{(df_valido['ANEMIA']==1).sum() / len(df_valido) * 100:.2f}%"
        ],
        'Sin_Anemia': [
            (df_train['ANEMIA']==0).sum(),
            (df_test['ANEMIA']==0).sum(),
            (df_dev['ANEMIA']==0).sum(),
            (df_valido['ANEMIA']==0).sum()
        ],
        'Pct_Sin_Anemia': [
            f"{(df_train['ANEMIA']==0).sum() / len(df_train) * 100:.2f}%",
            f"{(df_test['ANEMIA']==0).sum() / len(df_test) * 100:.2f}%",
            f"{(df_dev['ANEMIA']==0).sum() / len(df_dev) * 100:.2f}%",
            f"{(df_valido['ANEMIA']==0).sum() / len(df_valido) * 100:.2f}%"
        ]
    })
    stats_particion.to_excel(writer, sheet_name='Particion', index=False)
    
    # Hoja 4: Distribución proporcional por año en cada partición
    dist_años = []
    for año in sorted(df_valido['ANIO'].unique()):
        # Total por año
        n_total_año = len(df_valido[df_valido['ANIO'] == año])
        
        # Por partición
        n_train = len(df_train[df_train['ANIO'] == año])
        n_test = len(df_test[df_test['ANIO'] == año])
        n_dev = len(df_dev[df_dev['ANIO'] == año])
        
        # Porcentajes
        pct_train = n_train / n_total_año * 100 if n_total_año > 0 else 0
        pct_test = n_test / n_total_año * 100 if n_total_año > 0 else 0
        pct_dev = n_dev / n_total_año * 100 if n_total_año > 0 else 0
        
        dist_años.append({
            'Año': int(año),
            'Total_Año': n_total_año,
            'TRAIN': n_train,
            'TRAIN_%': f"{pct_train:.1f}%",
            'TEST': n_test,
            'TEST_%': f"{pct_test:.1f}%",
            'DEV': n_dev,
            'DEV_%': f"{pct_dev:.1f}%",
            'Verificacion': n_train + n_test + n_dev
        })
    
    df_dist_años = pd.DataFrame(dist_años)
    df_dist_años.to_excel(writer, sheet_name='Distribucion_Anual', index=False)
    
    # Hoja 5: Estratificación detallada (año x anemia x partición)
    estratos_detalle = []
    
    for año in sorted(df_valido['ANIO'].unique()):
        # Original
        df_año_orig = df_valido[df_valido['ANIO'] == año]
        n_orig = len(df_año_orig)
        n_anemia_orig = (df_año_orig['ANEMIA'] == 1).sum()
        pct_anemia_orig = n_anemia_orig / n_orig * 100 if n_orig > 0 else 0
        
        # TRAIN
        df_año_train = df_train[df_train['ANIO'] == año]
        n_train = len(df_año_train)
        n_anemia_train = (df_año_train['ANEMIA'] == 1).sum()
        pct_anemia_train = n_anemia_train / n_train * 100 if n_train > 0 else 0
        
        # TEST
        df_año_test = df_test[df_test['ANIO'] == año]
        n_test = len(df_año_test)
        n_anemia_test = (df_año_test['ANEMIA'] == 1).sum()
        pct_anemia_test = n_anemia_test / n_test * 100 if n_test > 0 else 0
        
        # DEV
        df_año_dev = df_dev[df_dev['ANIO'] == año]
        n_dev = len(df_año_dev)
        n_anemia_dev = (df_año_dev['ANEMIA'] == 1).sum()
        pct_anemia_dev = n_anemia_dev / n_dev * 100 if n_dev > 0 else 0
        
        estratos_detalle.append({
            'Año': int(año),
            'Original_Total': n_orig,
            'Original_Anemia%': f"{pct_anemia_orig:.2f}%",
            'TRAIN_Total': n_train,
            'TRAIN_Anemia%': f"{pct_anemia_train:.2f}%",
            'TRAIN_Diff': f"{abs(pct_anemia_train - pct_anemia_orig):.2f}pp",
            'TEST_Total': n_test,
            'TEST_Anemia%': f"{pct_anemia_test:.2f}%",
            'TEST_Diff': f"{abs(pct_anemia_test - pct_anemia_orig):.2f}pp",
            'DEV_Total': n_dev,
            'DEV_Anemia%': f"{pct_anemia_dev:.2f}%",
            'DEV_Diff': f"{abs(pct_anemia_dev - pct_anemia_orig):.2f}pp"
        })
    
    df_estratos = pd.DataFrame(estratos_detalle)
    df_estratos.to_excel(writer, sheet_name='Estratificacion_Detalle', index=False)

print(f"\n✓ Resumen:  {archivo_resumen}")

# ============================================================
# RESUMEN FINAL
# ============================================================
print("\n" + "="*80)
print("✅ PROCESO COMPLETADO")
print("="*80)

print(f"\n📁 Archivos generados en: {ruta_salida}")
print(f"   1. endes_train_2015_2024.csv ({len(df_train):,} registros)")
print(f"   2. endes_test_2015_2024.csv  ({len(df_test):,} registros)")
print(f"   3. endes_dev_2015_2024.csv   ({len(df_dev):,} registros)")
print(f"   4. endes_2015_2024_consolidado.csv ({len(df_consolidado):,} registros)")
print(f"   5. resumen_particion_endes.xlsx (5 hojas)")

print(f"\n🎯 Uso recomendado:")
print(f"   - TRAIN: Entrenar modelos (Random Forest, XGBoost, LightGBM)")
print(f"   - TEST:  Optimizar hiperparámetros y comparar modelos")
print(f"   - DEV:   Validación final del modelo seleccionado")

print(f"\n💡 Ventajas de esta partición:")
print(f"   ✓ Muestreo estratificado cruzado: AÑO × ANEMIA")
print(f"   ✓ Cada partición tiene TODOS los años (2015-2024)")
print(f"   ✓ Proporción de anemia se mantiene por año")
print(f"   ✓ Balance de clases similar entre TRAIN/TEST/DEV")
print(f"   ✓ Representatividad temporal en todas las particiones")

print(f"\n📊 Balance de ANEMIA conseguido:")
print(f"   GLOBAL: {prop_anemia_global:.2f}%")
print(f"   TRAIN:  {prop_anemia_train:.2f}% (Δ: {abs(prop_anemia_train - prop_anemia_global):.2f}pp)")
print(f"   TEST:   {prop_anemia_test:.2f}% (Δ: {abs(prop_anemia_test - prop_anemia_global):.2f}pp)")
print(f"   DEV:    {prop_anemia_dev:.2f}% (Δ: {abs(prop_anemia_dev - prop_anemia_global):.2f}pp)")

print(f"\n🎲 Estratificación cruzada exitosa:")
print(f"   ✅ Cada año mantiene su proporción original de anemia")
print(f"   ✅ Particiones 70/20/10 respetadas")
print(f"   ✅ Random state = 42 (reproducible)")

PROCESAMIENTO COMPLETO ENDES 2015-2024
GENERACIÓN DE BASES: TRAIN / TEST / DEV

PROCESANDO TODOS LOS AÑOS

PROCESANDO AÑO 2015

1. Cargando bases de datos...
   ✓ RECH0     : 36,692 registros,  10 columnas
      ⚠️  No encontradas: HV201, HV205, HV206
   ✓ RECH1     : 151,267 registros,   3 columnas
      ⚠️  No encontradas: HV009
   ✓ RECH6     : 25,527 registros,   9 columnas
   ✓ RECH23    : 36,692 registros,   2 columnas
   ✓ REC0111   : 36,655 registros,   6 columnas
   ✓ REC21     : 74,559 registros,   4 columnas
   ✓ REC44     : 23,821 registros,   9 columnas

2. Limpiando identificadores...
   ✓ RECH6: CASEID construido, ANIO agregado

3. Aplicando filtros metodológicos...
   RECH6: 25,527 → 12,552 (edad 6-35 meses, medición válida)
   RECH1: 151,267 → 143,228 (durmió noche anterior)
   RECH0: 36,692 → 34,595 (cuestionario completo)

4. Realizando joins...
   Base inicial: 12,552 registros
   ✓ JOIN 1: RECH1 | Matched: 12,183 (97.1%)
   ✓ JOIN 2: RECH0 | Matched: 12,552 (100.0%